In [78]:
## Here, I want to test the relaxation before and after the relaxation for the test calculations
## to calculate their spacegroup I can use pymatgen, seekpath and perhaps spglib. 

In [99]:
from aiida.orm import load_node, load_group
import aiida 
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import seekpath
from collections import defaultdict
import spglib

aiida.load_profile()

Profile<uuid='960a408e945248238ac64a4982cfc6d7' name='dev'>

In [100]:
def get_spacegroup_number_seekpath(structure_node):

    structure_ase = structure_node.get_ase()
    lattice = structure_ase.cell.array
    positions = structure_ase.get_scaled_positions()
    numbers = structure_ase.get_atomic_numbers()
    
    structure_spglib = (lattice, positions, numbers)
    
    kpath = seekpath.get_explicit_k_path(structure=structure_spglib)
    
    return kpath['spacegroup_number']


def get_spacegroup_number_spglib(structure_node):

    structure_ase = structure_node.get_ase()
    
    lattice = structure_ase.cell.array
    positions = structure_ase.get_scaled_positions()
    numbers = structure_ase.get_atomic_numbers()
    
    structure_spglib = (lattice, positions, numbers)
    
    spacegroup_data = spglib.get_spacegroup(structure_spglib)
    
    spacegroup_number = spacegroup_data.split()[1].replace('(','').replace(')','')  # The spacegroup number is the second element in the string
    
    return int(spacegroup_number)

In [101]:
## First we can check the workchain nodes that the calculations were succesfull.

group_name = 'workchain/test/relax'
group = load_group(group_name)

successful_nodes = []

for node in group.nodes:
    workchain_node = load_node(node.pk)
    if node.is_finished:
        if node.is_finished_ok:
            successful_nodes.append(node.pk)

In [102]:
##                      Pymatgen       Seekpath       Spglib
## Data[node(id)] = {[before,after],[before,after],[before,after]}
data = defaultdict()

In [103]:
## Pymatgen to do this test.

for node in successful_nodes: 
    
    workchain_node = load_node(node)
    
    structure_input = workchain_node.inputs.structure
    structure_input_pmg = structure_input.get_pymatgen()
    spganalyser_input = SpacegroupAnalyzer(structure_input_pmg,symprec=5e-6)
    spg_input_number = spganalyser_input.get_space_group_number()
    spg_input_operations_number = len(spganalyser_input.get_symmetry_operations())
    
    
    structure_output = workchain_node.outputs.output_structure
    structure_output_pmg = structure_output.get_pymatgen()
    spganalyser_output = SpacegroupAnalyzer(structure_output_pmg,symprec=5e-6)
    spg_output_number = spganalyser_output.get_space_group_number()
    spg_output_operations_number = len(spganalyser_output.get_symmetry_operations())
    
    if spg_output_number != spg_input_number: 
        
        print(node,spg_input_number,'->',spg_output_number)
        
    data[node] = [[spg_input_number,spg_output_number]]

In [104]:
## Seekpath to do this test. 

for node in successful_nodes: 
    
    workchain_node = load_node(node)
    
    structure_input = workchain_node.inputs.structure
    spacegroup_number_input = get_spacegroup_number_seekpath(structure_input)
    
    structure_output = workchain_node.outputs.output_structure
    spacegroup_number_output = get_spacegroup_number_seekpath(structure_output)
    
    data[node].append([spacegroup_number_input,spacegroup_number_output])

In [105]:
## Spglib to do this test. 

for node in successful_nodes:
    workchain_node = load_node(node)
    
    # Get the input structure and its space group number
    structure_input = workchain_node.inputs.structure
    spacegroup_number_input = get_spacegroup_number_spglib(structure_input)
    
    
    # Get the output structure and its space group number
    structure_output = workchain_node.outputs.output_structure
    spacegroup_number_output = get_spacegroup_number_spglib(structure_output)
    
    data[node].append([spacegroup_number_input,spacegroup_number_output])

In [106]:
## Check if seekpath and spglib agree. 

for key in data: 
   if not (data[key][0][0] == data[key][1][0] == data[key][2][0]):
       print('Input ',key,data[key][0][0],data[key][1][0],data[key][2][0]) 
       print('Output ', data[key][0][1],data[key][1][1],data[key][2][1]) 

Input  83580 129 67 67
Output  129 67 67


In [107]:
data

defaultdict(None,
            {74644: [[187, 187], [187, 187], [187, 187]],
             74684: [[8, 8], [8, 8], [8, 8]],
             74707: [[139, 139], [139, 139], [139, 139]],
             74733: [[139, 139], [139, 139], [139, 139]],
             74756: [[221, 221], [221, 221], [221, 221]],
             74783: [[187, 187], [187, 187], [187, 187]],
             74807: [[194, 194], [194, 194], [194, 194]],
             74831: [[141, 141], [141, 141], [141, 141]],
             74855: [[194, 194], [194, 194], [194, 194]],
             74879: [[221, 221], [221, 221], [221, 221]],
             74903: [[139, 139], [139, 139], [139, 139]],
             74951: [[139, 139], [139, 139], [139, 139]],
             74975: [[51, 51], [51, 51], [51, 51]],
             74999: [[194, 194], [194, 194], [194, 194]],
             75023: [[123, 123], [123, 123], [123, 123]],
             75047: [[51, 51], [51, 51], [51, 51]],
             75071: [[187, 187], [187, 187], [187, 187]],
             75095: 